In [17]:
import torch
import numpy as np

def load_obj(filename_obj, normalization=True, texture_size=4, load_texture=False,
             texture_wrapping='REPEAT', use_bilinear=True, use_cuda = False):
    """
    Load Wavefront .obj file.
    This function only supports vertices (v x x x) and faces (f x x x).
    """

    # read in all lines
    with open(filename_obj) as f:
        lines = f.readlines()

    # load vertices
    vertices = []
    for line in lines:
        if len(line.split()) == 0:
            continue
        if line.split()[0] == 'v':
            vertices.append([float(val) for val in line.split()[1:4]])
    vertices = torch.from_numpy(np.vstack(vertices).astype(np.float32))
    if use_cuda:
        vertices = vertices.cuda()

    # load vertex normal
    vertices_normal = []
    has_vn = False
    for line in lines:
        if len(line.split()) == 0:
            continue
        if line.split()[0] == 'vn':
            vertices_normal.append([float(val) for val in line.split()[1:4]])
    if vertices_normal:
        vertices_normal = torch.from_numpy(np.vstack(vertices_normal).astype(np.float32))
        if use_cuda:
            vertices_normal = vertices_normal.cuda()
        has_vn = True

    # load vertex texture coordinates
    vertices_texcoords = []
    has_vt = False
    for line in lines:
        if len(line.split()) == 0:
            continue
        if line.split()[0] == 'vt':
            vertices_texcoords.append([float(val) for val in line.split()[1:3]])
    if vertices_texcoords:
        vertices_texcoords = torch.from_numpy(np.vstack(vertices_texcoords).astype(np.float32))
        if use_cuda:
            vertices_texcoords = vertices_texcoords.cuda()
        has_vt = True

    # load faces
    faces = []
    faces_vn_idx = []
    faces_vt_idx = []
    face_mtl_idx = []
    face_mtl_names = []
    for line in lines:
        if len(line.split()) == 0:
            continue
            
        if line.split()[0] == 'usemap':
            face_mtl_names.append(line.split()[1])
            face_mtl_idx.append(len(faces))
            #print(line.split()[1])
            
        if line.split()[0] == 'f':
            vs = line.split()[1:]
            nv = len(vs)
            face = [int(vs[i].split('/')[0]) for i in range(nv)]
            faces.append(face)
            if has_vt:
                face_vt_idx = [int(vs[i].split('/')[1]) for i in range(nv)]
                faces_vt_idx.append(face_vt_idx)
            if has_vn:
                face_vn_idx = [int(vs[i].split('/')[-1]) for i in range(nv)]
                faces_vn_idx.append(face_vn_idx)
    faces = torch.from_numpy(np.vstack(faces).astype(np.int32)) - 1
    if len(faces_vn_idx)>0:
        faces_vn_idx = torch.from_numpy(np.vstack(faces_vn_idx).astype(np.int32)) - 1
    if len(faces_vt_idx)>0:
        faces_vt_idx = torch.from_numpy(np.vstack(faces_vt_idx).astype(np.int32)) - 1
    # face_mtl_idx = torch.from_numpy(np.vstack(face_mtl_idx).astype(np.int32))
    if use_cuda:
        faces = faces.cuda()
        faces_vn_idx = faces_vn_idx.cuda()
        faces_vt_idx = faces_vt_idx.cuda()

    # load textures
    textures = None
    if load_texture:
        for line in lines:
            if line.startswith('mtllib'):
                filename_mtl = os.path.join(os.path.dirname(filename_obj), line.split()[1])
                textures = load_textures(filename_obj, filename_mtl, texture_size,
                                         texture_wrapping=texture_wrapping,
                                         use_bilinear=use_bilinear)
        if textures is None:
            raise Exception('Failed to load textures.')

    # normalize into a unit cube centered zero
    #if normalization:
    #    vertices -= vertices.min(0)[0][None, :]
    #    vertices /= torch.abs(vertices).max()
    #    vertices *= 2
    #    vertices -= vertices.max(0)[0][None, :] / 2

    v_attr = {'v' : vertices, 'vn' : vertices_normal, 'vt' : vertices_texcoords}
    # f_mtl_idx: start idx of face for a mtl group
    # f_mtl_names: mtl group names
    f_attr = {'f_v_idx' : faces, 'f_vn_idx' : faces_vn_idx, 'f_vt_idx' : faces_vt_idx}
    # f_attr = {'f_v_idx' : faces, 'f_vn_idx' : faces_vn_idx, 'f_vt_idx' : faces_vt_idx, 'f_mtl_idx': face_mtl_idx, 'f_mtl_names': face_mtl_names}

    if load_texture:
        return v_attr, f_attr, textures 
    else:
        return v_attr, f_attr


In [18]:
import os
def save_obj(filename, v_attr, f_attr):
    vertices = v_attr['v']
    vertices_normal = v_attr['vn']
    vertices_textures = v_attr['vt']
    faces = f_attr['f_v_idx']
    assert vertices.ndimension() == 2
    assert faces.ndimension() == 2

    #f textures is not None:
    filename_mtl = 'smpl_white.mtl'
    filename_texture = 'SMPL_sampleTex_f.jpg'
    material_name = 'material_1'
    #texture_image, vertices_textures = create_texture_image(textures)
    #imsave(filename_texture, texture_image)

    faces = faces.detach().cpu().numpy()

    with open(filename, 'w') as f:
        f.write('# %s\n' % os.path.basename(filename))
        f.write('#\n')
        f.write('\n')

        f.write('mtllib %s\n\n' % os.path.basename(filename_mtl))

        for vertex in vertices:
            f.write('v %.8f %.8f %.8f\n' % (vertex[0], vertex[1], vertex[2]))
        f.write('\n')

        for normal in vertices_normal:
            f.write('vn %.8f %.8f %.8f\n' % (normal[0], normal[1], normal[2]))
        f.write('\n')
        
        for vertex in vertices_textures.reshape((-1, 2)):
            f.write('vt %.8f %.8f\n' % (vertex[0], vertex[1]))
        f.write('\n')

        f.write('usemtl %s\n' % material_name)
        for face, faces_vt_idx,faces_vn_idx in zip(f_attr['f_v_idx'], f_attr['f_vt_idx'], f_attr['f_vn_idx']):
            f.write('f %d/%d/%d %d/%d/%d %d/%d/%d\n' % (               \
                 face[0]+ 1, faces_vt_idx[0] + 1, faces_vn_idx[0] + 1, \
                 face[1]+ 1, faces_vt_idx[1] + 1, faces_vn_idx[1] + 1, \
                 face[2]+ 1, faces_vt_idx[2] + 1, faces_vn_idx[2] + 1))
        f.write('\n')

    with open(filename_mtl, 'w') as f:
        f.write('newmtl %s\n' % material_name)
        f.write('map_Kd %s\n' % os.path.basename(filename_texture))


In [19]:
# mesh lab function
from pathlib import *
def meshlab_resave(modelname, outputname):
    #odelname = Path(modelname)
    # meshlabserver = "D:\\MeshLab\\meshlabserver.exe"
    scriptname = " -s compute_v_normal.mlx"
    # output_params = " -m vc"
    meshlabserver = "meshlabserver"
    
    cmd = meshlabserver + " -i " + str(modelname) + scriptname + " -o " + str(outputname) + " -om vn"
    print(cmd)
    os.system(cmd)

In [28]:
# # rename image
# path = '/data/NFS/new_disk/chenxin/relightable-nr/data/realdome_cx/rgb0/'
# image_pattern = 'image.cam%02d_%06d.png'
# new_pattern = '%03d.png'
# for iF in range(20,151):
#     for iC in range(1,81):
#         orig_path = path + str(iF) + '/' + image_pattern%(iC,iF)
#         new_path = path + str(iF) + '/' + new_pattern%(iC-1)
#         print(orig_path)
#         print(new_path)
#         os.rename(orig_path, new_path)

# move obj
path = '/data/NFS/new_disk/chenxin/relightable-nr/data/white_walkshow'
# obj_pattern = '/%d/3DDeformed/%d_nonrigidRefinded_test0327_13.ply'
new_obj_pattern = '/mesh_smpl2pifu/aligned_%03d_017_512.ply'
rs_obj_pattern =  '/mesh_smpl2pifu_uv/aligned_%03d_017_512.obj'
for iF in range(90,95):
    # obj_path = path + obj_pattern%(iF,iF)
    new_path = path + new_obj_pattern%(iF)
    rs_path = path + rs_obj_pattern%(iF)
    #meshlab_resave(obj_path, new_path)
    meshlab_resave(new_path, rs_path)    

meshlabserver -i /data/NFS/new_disk/chenxin/relightable-nr/data/white_walkshow/mesh_smpl2pifu/aligned_090_017_512.ply -s compute_v_normal.mlx -o /data/NFS/new_disk/chenxin/relightable-nr/data/white_walkshow/mesh_smpl2pifu_uv/aligned_090_017_512.obj -om vn
meshlabserver -i /data/NFS/new_disk/chenxin/relightable-nr/data/white_walkshow/mesh_smpl2pifu/aligned_091_017_512.ply -s compute_v_normal.mlx -o /data/NFS/new_disk/chenxin/relightable-nr/data/white_walkshow/mesh_smpl2pifu_uv/aligned_091_017_512.obj -om vn
meshlabserver -i /data/NFS/new_disk/chenxin/relightable-nr/data/white_walkshow/mesh_smpl2pifu/aligned_092_017_512.ply -s compute_v_normal.mlx -o /data/NFS/new_disk/chenxin/relightable-nr/data/white_walkshow/mesh_smpl2pifu_uv/aligned_092_017_512.obj -om vn
meshlabserver -i /data/NFS/new_disk/chenxin/relightable-nr/data/white_walkshow/mesh_smpl2pifu/aligned_093_017_512.ply -s compute_v_normal.mlx -o /data/NFS/new_disk/chenxin/relightable-nr/data/white_walkshow/mesh_smpl2pifu_uv/aligned

In [29]:
# load reference
r_v_attr,r_f_attr = load_obj('/data/NFS/new_disk/chenxin/relightable-nr/data/white_walkshow/mesh_uv/smpl_test.obj')

# merge tex
for im in range(91,156):
    #filename = '/data/NFS/new_disk/chenxin/relightable-nr/data/white_walkshow/mesh_smpl2pifu/aligned_001_017_512.obj'%(im)
    filename_save = '/data/NFS/new_disk/chenxin/relightable-nr/data/white_walkshow/mesh_smpl2pifu_uv/aligned_%03d_017_512.obj'%(im)
    filename = filename_save
    v_attr, f_attr = load_obj(filename)
    v_attr['vt'] = r_v_attr['vt']
    
    f_attr['f_vt_idx'] = r_f_attr['f_vt_idx']

    print(filename)
    save_obj(filename_save, v_attr, f_attr)


/data/NFS/new_disk/chenxin/relightable-nr/data/white_walkshow/mesh_smpl2pifu_uv/aligned_091_017_512.obj
/data/NFS/new_disk/chenxin/relightable-nr/data/white_walkshow/mesh_smpl2pifu_uv/aligned_092_017_512.obj
/data/NFS/new_disk/chenxin/relightable-nr/data/white_walkshow/mesh_smpl2pifu_uv/aligned_093_017_512.obj
/data/NFS/new_disk/chenxin/relightable-nr/data/white_walkshow/mesh_smpl2pifu_uv/aligned_094_017_512.obj
/data/NFS/new_disk/chenxin/relightable-nr/data/white_walkshow/mesh_smpl2pifu_uv/aligned_095_017_512.obj
/data/NFS/new_disk/chenxin/relightable-nr/data/white_walkshow/mesh_smpl2pifu_uv/aligned_096_017_512.obj
/data/NFS/new_disk/chenxin/relightable-nr/data/white_walkshow/mesh_smpl2pifu_uv/aligned_097_017_512.obj
/data/NFS/new_disk/chenxin/relightable-nr/data/white_walkshow/mesh_smpl2pifu_uv/aligned_098_017_512.obj
/data/NFS/new_disk/chenxin/relightable-nr/data/white_walkshow/mesh_smpl2pifu_uv/aligned_099_017_512.obj
/data/NFS/new_disk/chenxin/relightable-nr/data/white_walkshow/me

In [ ]:
# reference

for im in range(160,350):
    filename = '.\..\obj\%03d.obj'%(im)
    filename_save = './../obj_merge/\%03d.obj'%(im)

    #filename = '160.obj'
    
    print('load mesh: ' + filename)
    v_attr, f_attr = load_obj(filename)
    fs_length = f_attr['f_v_idx'].shape[0]

    print('merge vertex coordinate ...')
    for i in range(0,f_attr['f_mtl_idx'].shape[0]):
        mtl_g = [];
        if i+1 < f_attr['f_mtl_idx'].shape[0]:
            mtl_g = range(f_attr['f_mtl_idx'][i],f_attr['f_mtl_idx'][i+1]);
        else:
            mtl_g = range(f_attr['f_mtl_idx'][i],fs_length);

        # merge vt            
        faces = np.reshape(f_attr['f_v_idx'][mtl_g],[-1,1]).tolist() # get face indx    
        if f_attr['f_mtl_names'][i] == 'surfaceShader_cloth':
            v_attr['vt'][faces] = v_attr['vt'][faces] * torch.from_numpy(np.array([0.8, 1]).astype(np.float32))
        if f_attr['f_mtl_names'][i] == 'surfaceShader_head':
            v_attr['vt'][faces] = torch.from_numpy(np.array([0.8, 0.75]).astype(np.float32)) + \
                                  v_attr['vt'][faces] * torch.from_numpy(np.array([0.2, 0.25]).astype(np.float32))
        if f_attr['f_mtl_names'][i] == 'surfaceShader_eye':
            v_attr['vt'][faces] = torch.from_numpy(np.array([0.8, 0.5]).astype(np.float32)) + \
                                  v_attr['vt'][faces] * torch.from_numpy(np.array([0.2, 0.25]).astype(np.float32))
        if f_attr['f_mtl_names'][i] == 'surfaceShader_body':
            v_attr['vt'][faces] = torch.from_numpy(np.array([0.8, 0.25]).astype(np.float32)) + \
                                  v_attr['vt'][faces] * torch.from_numpy(np.array([0.2, 0.25]).astype(np.float32))
        if f_attr['f_mtl_names'][i] == 'surfaceShader_shoes':
            v_attr['vt'][faces] = torch.from_numpy(np.array([0.8, 0]).astype(np.float32)) + \
                                  v_attr['vt'][faces] * torch.from_numpy(np.array([0.2, 0.25]).astype(np.float32))
    save_obj(filename_save, v_attr, f_attr)
